<a href="https://colab.research.google.com/github/dmera/DataAnalystPortfolioProjects/blob/main/GEE_setup_%26_supervised_classification_with_Landsat_8_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
ee.Authenticate()  # This will prompt a link for you to authenticate.


In [2]:
ee.Initialize(project='earth-engine-tji')

In [3]:
#!pip install geemap

In [4]:
import geemap

In [5]:
# Create a map
Map = geemap.Map()
Map

# Define the bounding box for New York City and its surroundings (approximate)
roi = ee.Geometry.BBox(-74.25909, 40.477399, -73.700181, 40.917577)  # West, South, East, North coordinates

# Load the Landsat 8 dataset (Collection 2, Level 2)
landsat = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
  .filterDate('2019-01-01', '2020-01-01') \
  .filterBounds(roi)

# Create a median composite of the image collection
composite = landsat.median()

# Compute the statistics for the bands (min/max)
stats = composite.reduceRegion(**{
    'reducer': ee.Reducer.minMax(),
    'geometry': roi,
    'scale': 30,
    'maxPixels': 1e9
})

# Get the dynamic min/max for the RGB bands (B4, B3, B2) and NIR bands
rgb_min = stats.get('SR_B4_min').getInfo(), stats.get('SR_B3_min').getInfo(), stats.get('SR_B2_min').getInfo()
rgb_max = stats.get('SR_B4_max').getInfo(), stats.get('SR_B3_max').getInfo(), stats.get('SR_B2_max').getInfo()

nir_min = stats.get('SR_B5_min').getInfo(), stats.get('SR_B4_min').getInfo(), stats.get('SR_B3_min').getInfo()
nir_max = stats.get('SR_B5_max').getInfo(), stats.get('SR_B4_max').getInfo(), stats.get('SR_B3_max').getInfo()

# Visualization parameters using dynamically computed min/max values
rgbVis = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': rgb_min, 'max': [rgb_max[0] * .7, rgb_max[1] * .7, rgb_max[2] * .7]}
nirVis = {'bands': ['SR_B5', 'SR_B4', 'SR_B3'], 'min': nir_min, 'max': nir_max}

# Add layers to the map
Map.addLayer(composite.clip(roi), rgbVis, 'RGB')
Map.addLayer(composite.clip(roi), nirVis, 'Near InfraRed')

# Center the map on the region of interest
Map.centerObject(roi, 10)
Map


Map(center=[40.697357934748915, -73.97963550000048], controls=(WidgetControl(options=['position', 'transparent…

In [6]:
 # Define the bounding box for New York City and its surroundings as region of interest (ROI) (approximate)
roi = ee.Geometry.BBox(-74.25909, 40.477399, -73.700181, 40.917577)  # West, South, East, North coordinates

#Apply scaling factor
def applyScaleFactors(image):
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('SR_B.').multiply(0.00341802).add(149.0)
    return image.addBands(opticalBands, None, True) \
                .addBands(thermalBands, None, True)
#Load Landsat 8 data
image = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
  .filterDate('2019-01-01', '2019-12-31') \
  .filterBounds(roi) \
  .map(applyScaleFactors) \
  .sort('CLOUD_COVER') \
  .first ()
#Create NDWI
ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename(['ndvi'])

#Create MNDWI image
mndwi = image.normalizedDifference(['SR_B3', 'SR_B6']).rename(['mndwi'])

images = image.addBands(ndvi).addBands(mndwi)

visualization = {
    'bands' : ['SR_B4', 'SR_B3', 'SR_B2'],
    'min' : 0,
    'max' : 0.3
}

Map = geemap.Map()
Map.centerObject(image, 8)
Map.addLayer(image, visualization, 'True Color (432)')
Map

Map(center=[41.75797503406744, -74.42221751197731], controls=(WidgetControl(options=['position', 'transparent_…

In [7]:
lc = ee.ImageCollection('projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m_TS').filterDate('2019-01-01', '2019-12-31').mosaic().remap([1,2,4,5,7,8,9,10,11],[1,2,3,4,5,6,7,8,9]).rename('lc')

In [8]:
print(lc.getInfo())

{'type': 'Image', 'bands': [{'id': 'lc', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 1, 'max': 9}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


In [9]:
label = 'lc'
bands = ['SR_B1','SR_B2','SR_B3','SR_B4','SR_B5','SR_B7','ndvi','mndwi']

#Sample the imput imagery to get a FeatureCollection ef training data
sample = images.addBands(lc).sample(**{
    'region': image.geometry(),
    'scale': 30,
    'numPixels': 5000,
    'seed': 1,
    #'geometries': True
})

#Add random value field to the sample and split 80% - 20% to training - validation
sample = sample.randomColumn()
trainingSample = sample.filter('random <= 0.8')
validationSample = sample.filter('random >0.8')

#Train a Random Forest
trainedClassifier = ee.Classifier.smileRandomForest(10).train(**{
    'features' : trainingSample,
    'classProperty' : label,
    'inputProperties' : bands
})

#Classify the validation data
model = images.classify(trainedClassifier)

In [10]:
#Define a dict which will be used to make legend and visualize image on th emap
dict = {
    'names' : [
        'Water', 'Trees', 'Flooded Vegetation', 'Crops', 'Built Area', 'Bare Ground', 'Snow/Ice', 'Clouds', 'Rangeland'
    ],

    'colors' : [
        '#1A5BAB',
        "#358221",
        "#87D19E",
        "#FFDB5C",
        "#ED022A",
        "#EDE9E4",
        "#F2FAFF",
        "#C8C8C8",
        "#C6AD8D"
    ]
}

In [11]:
#Add land cover to map object
Map.addLayer(lc.clip(image.geometry()), {'min': 1, 'max': 9, 'palette': dict['colors']}, 'ESRI LULC 10m 2017')
Map.addLayer(model, {'min': 1, 'max': 9, 'palette': dict['colors']}, 'Land Cover')
Map.centerObject(image, 8)
Map

Map(center=[41.75797503406744, -74.42221751197731], controls=(WidgetControl(options=['position', 'transparent_…